# Import libraries and token access

In [0]:
import base64
import requests
consumerKey = "pj6ItXK1rtEJuasI5nffHfV66Iga"
consumerSecret = "zB0tGFvqbzxeXSHu28CLaz6vIbQa"
keySecret = (consumerKey + ":" + consumerSecret).encode('utf-8')
consumerKeySecretB64 = base64.b64encode(keySecret).decode('utf-8')
tokenResponse = requests.post("https://apistore.datasparkanalytics.com/token",
  data = { 'grant_type': 'client_credentials' },  
  headers = { 'Authorization': 'Basic ' + consumerKeySecretB64 })
token = tokenResponse.json()['access_token']

# DataSpark OD API - Data Extraction

In this example, we will specify our origin (People's Park) and our destination (Aljunied). This OD pair has been identified as one of the busiest route from running the api. We will utilise DataSpark's OD API to identify alternative routes with a single transition point in between the OD. 

## Data for Origin to Intermediaries

We will start by getting data pertaining our origin to its intermediaries. We will be extracting the unique agen count (traffic), total records (trips made), total duration and total distance travelled. 

In [0]:
import json 

queryBody = {
"date": "2017-10-30", "timeSeriesReference": "destination", 
             
"location": {"locationType": "locationHierarchyLevel","levelType": "origin_subzone","id": "ORSZ03"},
             
"queryGranularity": {"type": "period", "period": "P1D"}, 

"filter":{"type": "selector", "dimension": "dominant_mode", "value": "ROAD"},                    
          
                         
"dimensionFacets": ["destination_subzone_name"],

"aggregations": [
    {"metric": "unique_agents","type": "hyperUnique"},
    {"metric": "total_records", "type": "longSum"},
    {"metric": "sum_duration" ,"type": "longSum"}, 
    {"metric": "sum_distance" ,"type": "longSum"}] }

queryResponse = requests.post("https://apistore.datasparkanalytics.com:443/odmatrix/v3/query",
 data = json.dumps(queryBody),
 headers = {
   'Authorization': 'Bearer ' + token,
   'Content-Type': 'application/json'
 }
) 
# OTSZ03
ODResult = queryResponse.json()

In [5]:
# This is where we break the json file to a dataframe with its individual cols
from pandas.io.json import json_normalize
exportOI = json_normalize(ODResult)

,event.destination_subzone_name,event.hyperUnique_unique_agents,event.longSum_sum_distance,event.longSum_sum_duration,event.longSum_total_records,event.origin_subzone,timestamp
0,ADMIRALTY,40,354873,38054,40,ORSZ03,2017-10-30T00:00:00.000+08:00
1,AIRPORT ROAD,40,222007,42442,40,ORSZ03,2017-10-30T00:00:00.000+08:00
2,ALEXANDRA HILL,40,406946,71736,40,ORSZ03,2017-10-30T00:00:00.000+08:00
3,ALJUNIED,238,2288255,408101,235,ORSZ03,2017-10-30T00:00:00.000+08:00
4,ANAK BUKIT,174,1619319,302391,171,ORSZ03,2017-10-30T00:00:00.000+08:00
5,ANCHORVALE,76,1070648,151454,76,ORSZ03,2017-10-30T00:00:00.000+08:00
6,ANG MO KIO TOWN CENTRE,40,291002,38166,40,ORSZ03,2017-10-30T00:00:00.000+08:00
7,ANSON,40,48544,15943,40,ORSZ03,2017-10-30T00:00:00.000+08:00
8,BALESTIER,40,334378,74711,40,ORSZ03,2017-10-30T00:00:00.000+08:00
9,BANGKIT,173,2019953,301553,171,ORSZ03,2017-10-30T00:00:00.000+08:00


## Data for Intermediaries to Destination

Next, we will get data from the intermediaries to our destination (Aljunied)

In [8]:
queryBody = {
"date": "2017-10-29", "timeSeriesReference": "origin", 
    
 "dimensionFacets": ["origin_subzone_name"],
    
"location": {"locationType": "locationHierarchyLevel","levelType": "destination_subzone","id": "CLSZ06"},
    
"queryGranularity": {"type": "period", "period": "P1D"}, #P1D,PT1H
    
"filter": {"type": "and", 
           "fields": [{"type": "not", "field" : {"type":"selector", "dimension": "agent_gender", "value": "NA"}  }, 
                      {"type": "selector", "dimension": "dominant_mode", "value": "ROAD"}, 
                      {"type": "not", "field": {"type": "selector", "dimension": "purpose", "value": "NA"} }, 
                      {"type": "not", "field": {"type": "selector", "dimension": "agent_nationality", "value": "NA"} },
                     ]
          },
    
"aggregations": [
    {"metric": "unique_agents","type": "hyperUnique"},
    {"metric": "total_records", "type": "longSum"},
    {"metric": "sum_duration" ,"type": "longSum"}, 
    {"metric": "sum_distance" ,"type": "longSum"}]
} 

queryResponse = requests.post("https://apistore.datasparkanalytics.com:443/odmatrix/v3/query",
 data = json.dumps(queryBody),
 headers = {
   'Authorization': 'Bearer ' + token,
   'Content-Type': 'application/json'
 }
) 

ODResultAlj = queryResponse.json()
exportID = json_normalize(ODResultAlj)

,event.destination_subzone,event.hyperUnique_unique_agents,event.longSum_sum_distance,event.longSum_sum_duration,event.longSum_total_records,event.origin_subzone_name,timestamp
33,CLSZ06,54,1393593,93002,55,CHANGI AIRPORT,2017-10-29T00:00:00.000+08:00
42,CLSZ06,41,505967,67400,41,CHOA CHU KANG NORTH,2017-10-29T00:00:00.000+08:00
47,CLSZ06,85,1795170,402813,93,CLEMENTI CENTRAL,2017-10-29T00:00:00.000+08:00
76,CLSZ06,72,499083,73654,72,HONG KAH,2017-10-29T00:00:00.000+08:00
77,CLSZ06,53,400058,64993,53,HONG KAH NORTH,2017-10-29T00:00:00.000+08:00
90,CLSZ06,85,806206,143645,84,JURONG WEST CENTRAL,2017-10-29T00:00:00.000+08:00
122,CLSZ06,55,484877,61283,57,MARITIME SQUARE,2017-10-29T00:00:00.000+08:00
192,CLSZ06,57,407843,67030,57,TAMAN JURONG,2017-10-29T00:00:00.000+08:00
243,CLSZ06,135,1484544,201173,136,YUNNAN,2017-10-29T00:00:00.000+08:00


## Inner join to combine Data from Origin to Intermediaries and Data from Intermediaries to Destination

Next, we will attempt to combine both sets of data pulled from the previous sections. We will create a "shared" column which will act as our joining factor ('join_col'). This column is essentially our intermediary points We wil also create 2 new columns, average duration and average distance which are calculated by dividing total duration/distance over the total trips made per OD pair respectively. 

In [0]:
exportOI['join_col'] = exportOI['event.destination_subzone_name'] + exportOI['timestamp']

In [0]:
exportOI['avg_duration'] = exportOI['event.longSum_sum_duration']/exportOI['event.longSum_total_records']

In [0]:
exportOI['avg_distance'] = exportOI['event.longSum_sum_distance']/exportOI['event.longSum_total_records']

In [0]:
exportID['join_col'] = exportID['event.origin_subzone_name'] + exportID['timestamp']

In [0]:
exportID['avg_duration'] = exportID['event.longSum_sum_duration']/exportID['event.longSum_total_records']

In [0]:
exportID['avg_distance'] = exportID['event.longSum_sum_distance']/exportID['event.longSum_total_records']

In [0]:
combined = exportOI.set_index('join_col').join(exportID.set_index('join_col'), lsuffix="exportOI", rsuffix="exportID", how = "inner")

3 new columns are created after combining the data. 

*   Total average duration is the duration taken to travel a alternate route.
*   Total average distance is the total distance from origin-intermediary-destination. 
*   Total load is the number of people travelling this alternate route.



In [0]:
combined['total_avg_duration'] = combined['avg_durationexportID'] + combined['avg_durationexportOI']

combined['total_avg_distance'] = combined['avg_distanceexportID'] + combined['avg_distanceexportOI']

combined['total_load'] = (combined['event.hyperUnique_unique_agentsexportOI'] + combined['event.hyperUnique_unique_agentsexportID'])/2

# Transfering relevant data into a new dataframe for building the app

In the final step, we will a new dataframe which we will export as a csv file to be used for generating the dashboards. The dataframe will consist of the following columns: 

*   Intermediate - middle/transition point in alternate route 
*   Timestamp - the time which this route was taken 
*   Average time - time taken for this origin-intermediary-destination route 
*   Total load - number of people travelling this route 





In [0]:
import pandas as pd
data = pd.DataFrame()
data['intermediate'] = combined['event.destination_subzone_name']
data['timestamp'] = combined['timestampexportOI'] 
data['avg_time'] = combined['total_avg_duration']
data['avg_distance'] = combined['total_avg_distance']
data['total_load'] = combined['total_load']

In [0]:
data = data.sort_values(by = ["timestamp", "avg_time", "total_load"])

# Creating a geographical boundary in which an intermediate point should be in

In [28]:
PeopleParkLat = 1.283428
PeopleParkLong = 103.84452
AljuniedLat = 1.316432
AljuniedLong = 103.882906
allowableLatIncrease = abs((AljuniedLat-PeopleParkLat)/2)

allowableLongIncrease = abs((AljuniedLong-PeopleParkLong)/2)

#50% increase outwards 
permissibleLat = [PeopleParkLat-allowableLatIncrease, AljuniedLat+allowableLatIncrease]
permissibleLong = [PeopleParkLong-allowableLongIncrease, AljuniedLong+allowableLongIncrease]
print(allowableLatIncrease)
print(allowableLongIncrease)
print(permissibleLat)
print(permissibleLong)

0.016502000000000017
0.019193000000001348
[1.266926, 1.332934]
[103.825327, 103.902099]
